This notebook contains how to load in the pre-trained weights from some methods best model, along with some figures demonstrating the performance.

In [23]:

from pytorch_lightning import Trainer
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import cv2


import sys
sys.path.append('../vit_pytorch/')
sys.path.append('..')
from vit import ViT
from recorder import Recorder # import the Recorder and instantiate
#from dataloaders import *

from training_loop import ViT_Trainer
from resnet_training_loop import *
from resnet import *
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from argparse import ArgumentParser, Namespace
import os
import random
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from dataloader import get_CIFAR_data
%matplotlib inline

In [34]:
def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


model = ResNet152()

In [2]:
base_best_model_path = '../vit_pytorch/lightning_logs/version_6668335/checkpoints/epoch=130.ckpt'
#best_model_path = '/scratch/nsk367/SP21/vit-pytorch/vit_pytorch/lightning_logs/version_6680321/checkpoints/epoch=64.ckpt'
#best_CNN_model_path = '/scratch/nsk367/SP21/vit-pytorch/vit_pytorch/lightning_logs/version_6708167/checkpoints/epoch=36.ckpt'


In [3]:
# Init our model
model = ViT_Trainer.load_from_checkpoint(base_best_model_path)

#model = ViT_Trainer.load_from_checkpoint(best_model_path)
#model = CNN_Trainer.load_from_checkpoint(best_CNN_model_path)

Files already downloaded and verified


/scratch/nsk367/anaconda3/envs/vit/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
for batch in model.test_dataloader():
    break

In [5]:
model.architecture

'ViTBase'

In [6]:
model.batch_size

128

In [40]:
model = model.cuda()



In [ ]:
model

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [37]:


#count_parameters(model._ViT_Trainer__model)
count_parameters(model)

58341028

In [38]:
batch[0].shape

torch.Size([128, 3, 32, 32])

In [41]:
%timeit model(batch[0].cuda())

90.4 ms ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


128 image batch passed through ResNet 18 which is 11173962 parameters, takes 9.81 ms ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


128 image batch passed through ResNet 34 which is 21328292 parameters, takes 17.3 ms ± 7.29 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


128 image batch passed through ResNet 152 which is 58341028 parameters, takes 90.4 ms ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


128 image batch passed through ViT base, which is 85124362 parameters, takes 133 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


128 image batch passed through ViT large, which is 302457956 parameters, takes 450 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
print("Resnet18 runtime per params  = ", 9.81/11173962 )

Resnet18 runtime per params  =  8.779338966787251e-07


In [44]:
print("Resnet34 runtime per params  = ", 17.3/21328292 )

Resnet34 runtime per params  =  8.111291799643404e-07


In [45]:
print("Resnet152 runtime per params  = ", 90.4/58341028 )

Resnet152 runtime per params  =  1.5495098920780073e-06


In [46]:
print("ViT Base runtime per params  = ", 133/85124362 )

ViT Base runtime per params  =  1.5624199333206162e-06


In [47]:
print("ViT Large runtime per params  = ", 450/302457956 )

ViT Large runtime per params  =  1.4878100941738825e-06


While ViT performs worse, it scales much better than 

In [ ]:
# 27 times the parameters, 50 times the runtime

In [ ]:
# This scaling is not particularly helpful. Instead, let's try with vitBase now too

In [ ]:
plt.imshow(batch[0][0].numpy().transpose((1,2,0)))

In [ ]:
model = model.cuda()

In [ ]:
batch[0].shape

In [ ]:
model(batch[0].cuda()).argmax(dim=1)

In [ ]:
batch[1]

In [ ]:
img = batch[0][j]

In [ ]:
img.shape

In [ ]:
model.rec.attn.shape

In [ ]:
j = 128 # using the jth element from that batch 
attn_mat = model.rec.attn[j].cpu()
img = batch[0]
im = img[j].cpu().numpy().transpose(1,2,0)
attn_mat = torch.mean(attn_mat, dim=1) # average across heads 
# To account for residual connections, we add an identity matrix to the
# attention matrix and re-normalize the weights.
residual_att = torch.eye(attn_mat.size(1))
aug_att_mat = attn_mat + residual_att
aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)
# Recursively multiply the weight matrices
joint_attentions = torch.zeros(aug_att_mat.size())
joint_attentions[0] = aug_att_mat[0]
for n in range(1, aug_att_mat.size(0)):
    joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n-1])

# combines all the different layers which apply attention. 

# Attention from the output token to the input space.
v = joint_attentions[-1]
grid_size = int(np.sqrt(aug_att_mat.size(-1)))
mask = v[0, 1:].reshape(grid_size, grid_size).detach().numpy()
mask = cv2.resize(mask / mask.max(), (model.image_size,model.image_size))[..., np.newaxis]
result = (mask * im.astype("uint8"))

In [ ]:
plt.imshow(result)

In [ ]:
plt.imshow(mask)

In [ ]:
plt.imshow(im)

In [ ]:
class_name[batch[1][j].item()]

In [ ]:
class_name[model(batch[0].cuda()).argmax(dim=1)[j].item()]

In [ ]:
nb_classes = 10
class_name = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck',
}